In [2]:
addpath ("../files")
setenv('GNUTERM','qt')
format long

In [2]:
%%file ../files/hessian.m

function [H] = hessian(fname,x)
    h = 1e-05;
    n = length(x);
    H = zeros(n);
    fx = feval(fname,x);

    for i = 1:n
        x1 = x;
        x1(i) = x1(i) + h;
        f1 = feval(fname, x1);
        for j = i:n
            x2 = x;
            x2(j) = x2(j) + h;
            f2 = feval(fname, x2);

            x3 = x1;
            x3(j) = x3(j) + h;
            f3 = feval(fname, x3);
            
            H(i,j) = (f3 - f2 - f1 + fx)/(h^2);
            H(j,i) = H(i,j);
        end
    end
end

Created file '/Users/sergio_arnaud/Documents/ITAM/8vo/applied_analysis/Labs/files/hessian.m'.


In [ ]:
hessian('rosenbrock',[1,1]')

In [3]:
%%file ../files/MetodoBL.m

function [W,x,iter] = MetodoBL(fname, x, method='newton', tol=1.e-04, maxiter = 50, maxjter = 30, c1 = 0.1)

%{
    Método de búsqueda de línea con la primer condición de Wolfe
    usando máximo descenso.
    usando los métodos de descenso:
        1. Por coordenadas
        2. Máximo descenso
    Input:
        
        fname.- cadena de caracteres con el nombre de la función a minimizar.
        x    .- vector n-dimensional.
    
    Output:
        x    .- vector n-dimensional con la aproximación al mínimo local.
        iter .- contador con el número final de iteraciones externas.
%}
    
    iter = 0;        
    p = 1; s = -1;

    fx = feval(fname,x);
    g  = gradiente(fname,x);
    ng = norm(g);
    W  = [x]; 

    while ( ng > tol && iter < maxiter &&  norm(p)> 1.e-04)

        if (strcmp(method,'max'))
            p = -g;
        end

        if (strcmp(method,'newton'))
            H = hessian(fname,x);
            p = -H\g; 
        end

        %-----------------------------------------
        % Búsqueda de línea
        alfa  = 1;              % paso completo
        xt = x + alfa*p;        % primer punto de prueba
        f  = feval(fname,x);  % valor de la función
        f1 = feval(fname, xt);% valor de la función en el punto de prueba
        s  = p'*g;            % derivada direccional
        jter = 0;             % iteraciones internas

        if (abs(s) < 1.e-06)
            disp('No existe suficiente descenso  ')
            disp(sprintf('%2.0f  %2.10f',iter, s   ))
            iter = maxiter; 
        end

        while( (f1>f+alfa*c1*s) && jter < maxjter)  % búsqueda de línea
           alfa = alfa/2;
           xt = x + alfa*p;
           f1 = feval(fname, xt); %valor de la función en el punto de prueba
           jter = jter +1;
        end   
        % Fin de búsqueda de línea
        %----------------------------------------------------------------------

        W = [W x];
        x = x + alfa*p;  
        fx = feval(fname,x); 
        g = gradiente(fname,x);
        iter = iter + 1;
    end      

end

Created file '/Users/sergio_arnaud/Documents/ITAM/8vo/Applied_analysis/Labs/files/MetodoBL.m'.


In [4]:
[W,x,iter] = MetodoBL('rosenbrock',[0,0]');

In [4]:
function [fx] = rosenbrock1(x,y)
%{
    Función de Rosenbrock: f: R^2 --> R cuyo mínimo local es muy 
    difícil de alcanzar por medio de los métodos de optimización.
    
    Input:
         x.- vector de longitud 2
    Output:
        fx.- número real con el valor de la función.
%}
    fx = 100.*(y - x.^2).^2 + (1 - x).^2;
end

In [5]:
%plot gnuplot

x = linspace(-2,2,25);
y = linspace(-2,2,25);
[X,Y] = meshgrid(x,y);

hold on;
scatter3(W(1,:),W(2,:),rosenbrock1(W(1,:),W(2,:)),200,'fill')
surf(X,Y,rosenbrock1(X,Y),'FaceAlpha',0.4)
hold off;

error: 'W' undefined near line 1 column 10
